# Cloud-Edge network simulation

This example demonstrates how to estimate Cloud-Edge networks with different topologies and node interconnections.

Users can define network components and seamlessly interconnect them to create networks of varying sizes and topologies. The code is designed to handle the increasing complexity of networks, allowing users to extend and scale the software to support more intricate scenarios.

The flexibility of the code becomes apparent in the provided example, where users are guided on how to define network components, set parameters, and create networks tailored to their specific needs. This adaptability ensures that the software is not limited to a particular network topology or size, making it suitable for diverse applications.

Concerning the linkage of different system components, the code establishes clear and robust connections to ensure the desired functionality. The example demonstrates how these components interact, providing users with a comprehensive understanding of the underlying architecture.

Moreover, the software's extensibility is a key feature. As networks become more complex, users can easily extend the code to accommodate additional components and functionalities. The code's architecture is designed with scalability in mind, empowering users to handle a broad spectrum of use cases.

## Description of the network to be modelled

To simulate a network, it is necessary to understand how network nodes are interconnected and how data flows are created and transmitted across the network. For example, in the provided network structure, data flows are initiated by reading data from sensors. The data then undergoes processing, which can be executed by either Edge devices or Cloud servers. The simulation can be utilized to estimate how network performance depends on the distribution of data flows between Edge and Cloud processing devices. Additionally, it aids in determining the number of devices needed to meet waiting time requirements, estimating costs, and exploring various scenarios.

<img src="pics/Network5.svg" />

The parameters of data flow generation, processing and distribution are defined in following code block.

In [12]:
sensor_data_polling_rate = 30 # req/h (sensor polling rate = rate of data processing requests)
sensor_data_length = 8000*8 # bits
sensor_network_data_rate = 1_000_000 # bits/s
sensor_network_service_rate = 3600*sensor_network_data_rate/sensor_data_length # req/h
retinue_per_processed_data = 0.1 # Eur

edge_network_data_rate = 100_000_000 # bits/s
edge_network_service_rate = 3600*edge_network_data_rate/sensor_data_length # req/h
edge_device_data_processing_time = 200 # s
edge_device_battery_perf_index = 200 # number of data processing cycles to discharge a full battery (used to estimate how many hours Edge device may work on battery) 
edge_device_service_rate = 3600*1/edge_device_data_processing_time # req/h
edge_device_cost = 0.5 # Eur/h

cloud_computing_network_data_rate = 1_000_000_000 # bits/s
cloud_computing_network_service_rate = 3600*cloud_computing_network_data_rate/sensor_data_length # req/h
cloud_server_processing_time = 100 # s
cloud_server_service_rate = 3600*1/cloud_server_processing_time # req/h
cloud_server_cost = 0.15 # Eur/h

processed_data_length = 200*8 # bits
cloud_storage_network_data_rate = 1_000_000_000 # bits/s
cloud_database_data_write_rate = 200_000_000*8 # bits/s
cloud_storage_network_service_rate = 3600*cloud_storage_network_data_rate/processed_data_length # req/h 
cloud_database_service_rate = 3600*cloud_database_data_write_rate/processed_data_length # req/h

edge_load_ratio = 0.2
cloud_load_ratio = 1 - edge_load_ratio

## Usage of the "network" package

The "network" package was developed to define network nodes as distinct object classes and to leverage the same queuing systems models employed in other examples within the CloudEdgeAssetsOptimizer package.

All network node classes inherit from the general Node class, incorporating specific changes that pertain to individual network node features. Further details can be found in the comment lines within the network.py file.

The subsequent code block includes an example demonstrating how to create various network nodes, interconnect them, simulate data flow, and review the resulting parameters.

If the exception error occurs (Exception: Unstable system, because Arrival rate > Service rate. Ensure that: 's' < 'a' or 'ar' < 'sr'), it indicates that the intensity of the data flow from the sensors exceeds the data processing rate of Edge devices or Cloud servers. In such a case, it is recommended to either decrease the number of sensors or the data polling rate, or alternatively, increase the number of data processing Edge devices and Cloud servers.

In [13]:
# Imports contents of network package
from network import *

# Creates three Sensors with given id strings and data polling rate
sensor1 = Sensor("Sensor 1",sensor_data_polling_rate)
sensor2 = Sensor("Sensor 2",sensor_data_polling_rate)
sensor3 = Sensor("Sensor 3",sensor_data_polling_rate)

# Creates three Edge devices for sensor data processing with given parameters
edge_device1 = EdgeDevice("Edge 1",edge_device_service_rate,edge_device_battery_perf_index,edge_device_cost,retinue_per_processed_data)
edge_device2 = EdgeDevice("Edge 2",edge_device_service_rate,edge_device_battery_perf_index,edge_device_cost,retinue_per_processed_data)
edge_device3 = EdgeDevice("Edge 3",edge_device_service_rate,edge_device_battery_perf_index,edge_device_cost,retinue_per_processed_data)

# Creates three Cloud servers for sensor data processing with given parameters
cloud_server1 = CloudServer("Cloud Server 1",cloud_server_service_rate,cloud_server_cost,retinue_per_processed_data)
cloud_server2 = CloudServer("Cloud Server 2",cloud_server_service_rate,cloud_server_cost,retinue_per_processed_data)
cloud_server3 = CloudServer("Cloud Server 3",cloud_server_service_rate,cloud_server_cost,retinue_per_processed_data)

# Creates Edge gateway that performs load balancing between Edge and Cloud networks
balancer = Balancer("Edge Gateway")

# Creates Database that stores processed data results
database = Database("Database",cloud_database_service_rate)

# Creates Sensor, Edge, Cloud Computing and Cloud Storage network data channels
sensor_network = DataChannel("Sensor Network",sensor_network_service_rate)
edge_network = DataChannel("Edge Network",edge_network_service_rate)
cloud_network = DataChannel("Cloud Computing Network",cloud_computing_network_service_rate)
cloud_storage_network = DataChannel("Cloud Storage Network",cloud_storage_network_service_rate)

# Connects sensors to the Sensor network
sensor1.connect_to(sensor_network)
sensor2.connect_to(sensor_network)
sensor3.connect_to(sensor_network)

# Connects Sensor network to the Edge gateway (load balancer)
sensor_network.connect_to(balancer)

# Connects Edge gateway (load balancer) to the Edge and Cloud networks with given load distribution ratios
balancer.connect_to(edge_network,edge_load_ratio)
balancer.connect_to(cloud_network,cloud_load_ratio)

# Connects Edge network to the Edge devices with given load distribution ratios
# (ratios may be different, but their sum must be equal to 1)
edge_network.connect_to(edge_device1,ratio=2/9)
edge_network.connect_to(edge_device2,ratio=3/9)
edge_network.connect_to(edge_device3,ratio=4/9)

# Connects Cloud network to the Cloud servers with given load distribution ratios
cloud_network.connect_to(cloud_server1,ratio=2/9)
cloud_network.connect_to(cloud_server2,ratio=4/9)
cloud_network.connect_to(cloud_server3,ratio=3/9)

# Connects all data processing nodes to the Cloud storage network
edge_device1.connect_to(cloud_storage_network)
edge_device2.connect_to(cloud_storage_network)
edge_device3.connect_to(cloud_storage_network)
cloud_server1.connect_to(cloud_storage_network)
cloud_server2.connect_to(cloud_storage_network)
cloud_server3.connect_to(cloud_storage_network)

# Connects Cloud storage network to the Database
cloud_storage_network.connect_to(database)

# The get_info() function can be used to review how data is transferred from Sensors to the Database (last node)  
print("sensor1.get_info() results:")
sensor1.get_info()

# To review all the network node parameters the print(node) function can be used, for example
print("\nprint(edge_device1) results:")
print(edge_device1)

# To obtain value of particular parameter node.parameter syntax is used, for example
print("\nprint(edge_device1.battery_time) results:")
print(edge_device1.battery_time)


sensor1.get_info() results:
Sensor 1 --> Sensor Network --> Edge Gateway --> Edge Network --> Edge 1 --> Cloud Storage Network --> Database, Total time: 228.577304, Total profit: -0.100641
Sensor 1 --> Sensor Network --> Edge Gateway --> Edge Network --> Edge 2 --> Cloud Storage Network --> Database, Total time: 249.944647, Total profit: 0.099039
Sensor 1 --> Sensor Network --> Edge Gateway --> Edge Network --> Edge 3 --> Cloud Storage Network --> Database, Total time: 279.834314, Total profit: 0.298718
Sensor 1 --> Sensor Network --> Edge Gateway --> Cloud Computing Network --> Cloud Server 1 --> Cloud Storage Network --> Database, Total time: 139.949065, Total profit: 1.447440
Sensor 1 --> Sensor Network --> Edge Gateway --> Cloud Computing Network --> Cloud Server 2 --> Cloud Storage Network --> Database, Total time: 494.376921, Total profit: 3.044880
Sensor 1 --> Sensor Network --> Edge Gateway --> Cloud Computing Network --> Cloud Server 3 --> Cloud Storage Network --> Database, T

The subsequent code block illustrates how to modify the given example to generate a desired number of Sensors, Edge devices, and Cloud servers. In this particular case, the load from Edge Network to Edge devices is evenly distributed, and the load from Cloud Network to Cloud Servers is also evenly distributed. That's why the resulting parameters are identical for all Edge devices and the same for Cloud servers.

In [14]:
from network import *

# Number of Sensors, Edge devices and Cloud servers is defined:
sensors_no = 9
edge_devices_no = 8
cloud_servers_no = 6

# Load distribution between Edge network and Cloud network is defined:
edge_load_ratio = 0.4
cloud_load_ratio = 1 - edge_load_ratio

# Lists of sensors, edge_devices and cloud_servers are created:
sensors = []
for s_no in range(1,sensors_no+1):
    sensors.append(Sensor("Sensor %d"%s_no,sensor_data_polling_rate))

edge_devices = []
for e_no in range(1,edge_devices_no+1):
    edge_devices.append(EdgeDevice("Edge %d"%e_no,edge_device_service_rate,edge_device_battery_perf_index,edge_device_cost,retinue_per_processed_data))

cloud_servers = []
for c_no in range(1,cloud_servers_no+1):
    cloud_servers.append(CloudServer("Cloud Server %d"%c_no,cloud_server_service_rate,cloud_server_cost,retinue_per_processed_data))

# Everything is interconnected according the simulated network architecture 
sensor_network = DataChannel("Sensor Network",sensor_network_service_rate)
edge_network = DataChannel("Edge Network",edge_network_service_rate)
cloud_network = DataChannel("Cloud Computing Network",cloud_computing_network_service_rate)

balancer = Balancer("Edge Gateway")

cloud_storage_network = DataChannel("Cloud Storage Network",cloud_storage_network_service_rate)
database = Database("Database",cloud_database_service_rate)

for sensor in sensors:
    sensor.connect_to(sensor_network)

sensor_network.connect_to(balancer)
balancer.connect_to(edge_network,edge_load_ratio)
balancer.connect_to(cloud_network,cloud_load_ratio)

for edge_device in edge_devices:
    edge_network.connect_to(edge_device,1/edge_devices_no) # 1/edge_devices_no - equal distribution ratio

for cloud_server in cloud_servers:
    cloud_network.connect_to(cloud_server,1/cloud_servers_no)

for edge_device in edge_devices:
    edge_device.connect_to(cloud_storage_network)

for cloud_server in cloud_servers:
    cloud_server.connect_to(cloud_storage_network)

cloud_storage_network.connect_to(database)


# Example how to obtain data flow info for sensors[0] Sensor
sensors[0].get_info()

# Example how to calculate the total profit of the simulate Edge-Cloud data processing network
total_profit = 0
for edge_device in edge_devices:
    total_profit += edge_device.profit
for cloud_server in cloud_servers:
    total_profit += cloud_server.profit

print("Total profit", total_profit)


Sensor 1 --> Sensor Network --> Edge Gateway --> Edge Network --> Edge 1 --> Cloud Storage Network --> Database, Total time: 494.377757, Total profit: 0.843510
Sensor 1 --> Sensor Network --> Edge Gateway --> Edge Network --> Edge 2 --> Cloud Storage Network --> Database, Total time: 494.377757, Total profit: 0.843510
Sensor 1 --> Sensor Network --> Edge Gateway --> Edge Network --> Edge 3 --> Cloud Storage Network --> Database, Total time: 494.377757, Total profit: 0.843510
Sensor 1 --> Sensor Network --> Edge Gateway --> Edge Network --> Edge 4 --> Cloud Storage Network --> Database, Total time: 494.377757, Total profit: 0.843510
Sensor 1 --> Sensor Network --> Edge Gateway --> Edge Network --> Edge 5 --> Cloud Storage Network --> Database, Total time: 494.377757, Total profit: 0.843510
Sensor 1 --> Sensor Network --> Edge Gateway --> Edge Network --> Edge 6 --> Cloud Storage Network --> Database, Total time: 494.377757, Total profit: 0.843510
Sensor 1 --> Sensor Network --> Edge Gat

The simulated network can be adjusted by adding extra Sensors, Edge devices, Cloud servers, and sub-networks. Each connection and network node can be assigned unique parameters. By fine-tuning network, equipment, and data flow parameters, users can pinpoint the optimal configuration for specific requirements. If a new parameter is needed, the open-source code can be modified accordingly.